In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as Data
import matplotlib.pyplot as plt
from torchvision import transforms
import torchvision.models as models
import yaml
import datetime
import pandas as pd
from PIL import Image
import pickle
import math

In [2]:
LR = 0.5
BATCH_SIZE = 64
EPOCH = 100000

INDEX = []
TRAIN_ACCURANCY = []
TRAIN_LOSS = []
TEST_ACCURANCY = []
TEST_LOSS = []
ACCURANCY = 0.0

theTime = datetime.datetime.now().strftime("%Y%m%d_%H%M")

In [3]:
def getData(mode):
    if mode == 'train':
        img = pd.read_csv('set00_train_img.csv')
        label = pd.read_csv('set00_train_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)
    else:
        img = pd.read_csv('set00_test_img.csv')
        label = pd.read_csv('set00_test_label.csv')
        return np.squeeze(img.values), np.squeeze(label.values)


class RetinopathyLoader(Data.Dataset):
    def __init__(self, root, mode):
        """
        Args:
            root (string): Root path of the dataset.
            mode : Indicate procedure status(training or testing)

            self.img_name (string list): String list that store all image names.
            self.label (int or float list): Numerical list that store all ground truth label values.
        """
        self.root = root
        self.img_name, self.label = getData(mode)
        self.mode = mode
        print("> Found %d images..." % (len(self.img_name)))
        

    def __len__(self):
        """'return the size of dataset"""
        return len(self.img_name)

    def __getitem__(self, index):
        """something you should implement here"""

        """step1. Get the image path from 'self.img_name' and load it.
                  hint : path = root + self.img_name[index] + '.jpeg'"""
        path = self.root + self.img_name[index] + '.png'
        img = Image.open(path)
        img_as_img = img.resize(( 160, 120),Image.ANTIALIAS)
        
        """step2. Get the ground truth label from self.label"""
        label = torch.from_numpy(self.label)[index]
        """ step3. Transform the .jpeg rgb images during the training phase, such as resizing, random flipping, 
                  rotation, cropping, normalization etc. But at the beginning, I suggest you follow the hints. 
                       
                  In the testing phase, if you have a normalization process during the training phase, you only need 
                  to normalize the data. 
                  
                  hints : Convert the pixel value to [0, 1]
                          Transpose the image shape from [H, W, C] to [C, H, W]"""
        if self.mode == 'train':
            transform1 = transforms.Compose([
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.5),
                transforms.RandomRotation(degrees=(-45,45), resample=False, expand=False, center=None),
                transforms.ColorJitter(contrast=(0,1)),
                transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                ]
            )
        else:
            transform1 = transforms.Compose([
                transforms.ToTensor(), # range [0, 255] -> [0.0,1.0]
                ]
            )
        img_tran = transform1(img_as_img)
                
        """step4. Return processed image and label"""

        return img_tran, label

In [4]:
# Dataloader

train_data=RetinopathyLoader('./set00/','train')
test_data=RetinopathyLoader('./set00/','test')
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE)
test_loader = Data.DataLoader(dataset=test_data, batch_size=BATCH_SIZE)
print (train_loader)
print (test_loader)

> Found 2339 images...
> Found 259 images...


In [5]:
# if(self.act==1):
#             out = F.elu(out)   
#         elif(self.act==2):
#             out = F.relu(out)
#         else:
#             out = F.leaky_relu(out)
class DeepConvNet(nn.Module):
    def __init__(self,active_func):
        super(DeepConvNet, self).__init__()
    
        #####---1
        self.conv1 = nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=0, bias=False)
        self.batchnorm1 = nn.BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.norm1 = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=1.0)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        #####
        
        #####---2
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=(2, 2) ,bias=True, groups=2)
        self.batchnorm2 = nn.BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        self.norm2 = nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=1.0)
        self.pool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=0)
        #####
        
        #####---3
        self.conv3 = nn.Conv2d(256, 384, kernel_size=(3, 3), padding=(1, 1), bias=False)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=(3, 3), padding=(1, 1), bias=True, groups=2)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=(3, 3), padding=(1, 1), bias=True, groups=2)
        self.pool5 = nn.MaxPool2d(kernel_size=3 , stride=2, padding=0)
        #####
        
        self.fc6 = nn.Linear(in_features=1536, out_features=4096, bias=True)
        self.fc7 = nn.Linear(in_features=4096, out_features=4096, bias=True)
        self.fc8 = nn.Linear(in_features=4096, out_features=256, bias=False)
        self.fc9 = nn.Linear(in_features=256, out_features=2, bias=True)
        self.act=active_func
        
    def forward(self, x):
        
        ###-----1------
        out = self.conv1(x)
        #out = self.batchnorm1(out)
        out = F.relu(out)
        out = self.pool1(out)        
        out = self.norm1(out)
        #out = F.dropout(out, p=0.5)
        
        ###-----2------
        out = self.conv2(out)
        #out = self.batchnorm2(out)
        out = F.relu(out)
        out = self.pool2(out)
        out = self.norm2(out)
        #out = F.dropout(out, p=0.5)
        
        ###-----3------
        out = self.conv3(out)
        out = F.relu(out)
        ###-----4------
        out = self.conv4(out)
        out = F.relu(out)
        
        out = self.conv5(out)
        out = F.relu(out)
        out = self.pool5(out)
        
        out = out.view(out.size(0), -1) #flatten
        out = self.fc6(out)
        out = F.relu(out)
        out = F.dropout(out, p=0.5)
        
        out = self.fc7(out)
        out = F.relu(out)
        out = F.dropout(out, p=0.5)
      
        out = self.fc8(out)
        out = F.relu(out)
        out = F.dropout(out, p=0.5)
        
        out = self.fc9(out)
        out = F.sigmoid(out)
        
        return out

In [6]:
def train_DeepConv(epoch):
    model_DeepConvNet.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        if torch.cuda.is_available():
            data, target = data.to(device=device, dtype=torch.float), target.to(device)

        optimizer.zero_grad()
        output = model_DeepConvNet(data)
        target = target.float()
        loss = Loss(output, target)
        loss.backward()
        optimizer.step()
        #pred = output.data.max(1)[1] # get the index of the max log-probability
        #correct += pred.eq(target.data).cpu().sum()
    #TRAIN_ACCURANCY.append(round(100.00 * float(correct)/len(train_loader.dataset),2))
    TRAIN_LOSS.append(round((loss.data).cpu().numpy().tolist(),6))
   
    print('Train Epoch: {} \tLoss: {}'.format(
            epoch, loss.data))

In [7]:
def test_DeepConv(epoch):
    model_DeepConvNet.eval()
    test_loss = 0.0
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        if torch.cuda.is_available():
            data, target = data.to(device=device, dtype=torch.float), target.to(device)
        with torch.no_grad():
            output = model_DeepConvNet(data)
        target = target.float()
        test_loss += Loss(output, target).data
        #pred = output.data.max(1)[1] # get the index of the max log-probability
        #correct += pred.eq(target.data).cpu().sum()
    
    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    #TEST_ACCURANCY.append(round(100.00 * float(correct) / float(len(test_loader.dataset)),2))
    TEST_LOSS.append(round(test_loss.cpu().numpy().tolist(),4))
    print('---------------Test set: Average loss: {}-------------------\n'.format(test_loss))

In [8]:
def show_result(index,TRAIN_ACCURANCY_1,TEST_ACCURANCY_1,TRAIN_ACCURANCY_2,TEST_ACCURANCY_2,TRAIN_ACCURANCY_3,TEST_ACCURANCY_3):
    plt.figure(figsize=(13,7))
    plt.title('DeepConvNet',fontsize=20)
    plt.xlabel('epoches',fontsize=18)
    plt.ylabel('accurancy(%)',fontsize=18)
    plt.xlim((index[0]-25, index[len(index)-1]))
    plt.ylim((48, 100))
    plt.plot(index,TRAIN_ACCURANCY_1,'r-',label='Train_ELU')
    plt.plot(index,TEST_ACCURANCY_1,'b-',label='Test_ELU')
    plt.plot(index,TRAIN_ACCURANCY_2,'g-',label='Train_Relu')
    plt.plot(index,TEST_ACCURANCY_2,'y-',label='Test_Relu')
    plt.plot(index,TRAIN_ACCURANCY_3,'k-',label='Train_Leaky_relu')
    plt.plot(index,TEST_ACCURANCY_3,'m-',label='Test_Leaky_relu')
    plt.legend(loc='lower right',fontsize=12)
    plt.show()
    figfilename = './DConv_ELU_'+theTime+'.png'
    plt.savefig(figfilename)

In [ ]:
model_DeepConvNet = DeepConvNet(1).cuda(0)

if torch.cuda.is_available():
    device = torch.device('cuda')
    model_DeepConvNet.to(device) 
#------------define optimizer/loss function
Loss = nn.MSELoss(reduction='mean')    
optimizer = torch.optim.Adam(model_DeepConvNet.parameters(), lr=LR)
print(model_DeepConvNet)

DeepConvNet(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), bias=False)
  (batchnorm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm1): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (pool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2)
  (batchnorm2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm2): LocalResponseNorm(5, alpha=0.0001, beta=0.75, k=1.0)
  (pool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (pool5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilati

In [ ]:
INDEX = []
TRAIN_ACCURANCY = []
TRAIN_LOSS = []
TEST_ACCURANCY = []
TEST_LOSS = []
ACCURANCY = 0.0

for epoch in range(1, EPOCH + 1):
    INDEX.append(epoch)
    train_DeepConv(epoch)
    test_DeepConv(epoch)
    if (epoch%100==0):
        PATH =str(epoch)+'model.pkl'
        torch.save(model_DeepConvNet.state_dict(), PATH)
    '''
    savefilename = str(epoch)+'.tar'
    torch.save({
            'epoch': epoch,
            'state_dict': model_EEGNet.state_dict(),
        }, savefilename)
    '''

#return round(max(TEST_ACCURANCY),2), TRAIN_ACCURANCY, TEST_ACCURANCY

/home/lilichiu/.local/lib/python3.5/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 1 	Loss: 0.2114713042974472
---------------Test set: Average loss: 0.1920444518327713-------------------



In [ ]:

#     INDEX = []
#     TRAIN_ACCURANCY = []
#     TRAIN_LOSS = []
#     TEST_ACCURANCY = []
#     TEST_LOSS = []
#     ACCURANCY = 0.0
    
#     model_DeepConvNet = DeepConvNet(1).cuda(0)
#     if torch.cuda.is_available():
#         device = torch.device('cuda')
#         model_DeepConvNet.to(device) 
#     #------------define optimizer/loss function
#     Loss = nn.MSELoss()    
#     optimizer = torch.optim.Adam(model_DeepConvNet.parameters(), lr=LR)
#     train_1()
#     #---------------1:elu   2:relu   3:leakyrelu---------------------#
# #     ACCURANCY_1,TRAIN_ACCURANCY_1,TEST_ACCURANCY_1 = train_1()
# #     ACCURANCY_2,TRAIN_ACCURANCY_2,TEST_ACCURANCY_2 = train_2()
# #     ACCURANCY_3,TRAIN_ACCURANCY_3,TEST_ACCURANCY_3 = train_3()
    
# #     print("\t\t\t\tThe Hightest Test Accurancy:\n\t\t\t\t\tElu:",ACCURANCY_1,"\n\t\t\t\t\tRelu:",ACCURANCY_2,"\n\t\t\t\t\tLeaky Relu:",ACCURANCY_3)
# #     show_result(INDEX,TRAIN_ACCURANCY_1,TEST_ACCURANCY_1,TRAIN_ACCURANCY_2,TEST_ACCURANCY_2,TRAIN_ACCURANCY_3,TEST_ACCURANCY_3)
